In [59]:
# Setting up the environment
import simpy
import random
import statistics
import numpy as np

In [219]:
"""
M/M/c Simulation of Tenant Onboarding Stage.

Scenario:
    A customer has successfully submitted their ADO ticket to commence their tenant onboarding process.
    The customer has one on-call engineer deploy SF clusters on their behalf.
    The customer is fully onboarded to the service.

Goal: understand how much time the customer is being spent on queue delays
"""

RANDOM_SEED = 2022
ARRIVAL_TIME = 4        # in days (lambda in expovariate)
TIME_TO_COMPLETE = 14   # in days (mu)
SIM_TIME = 182          # ~half a year

class MonitoredResource(simpy.Resource):
    """MonitoredResource: custom class for monitoring
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = []
        self.total_service_time = 0.0
        self.customer_times = []
        self.wait_times = []

    def request(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue)))
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        self.data.append((self._env.now, len(self.queue)))
        return super().release(*args, **kwargs)


def arrival(env, interval, mu, resource):
    """Arrival process (entering tenant onboarding queue)
    Args:
        env: Simpy environment
        interval: interval of arrival (every x days)
        mu: time it takes to complete a tenant onboarding
        resource: Resource to use
    """
    i = 0
    while True:
        customer = serve(env, "Customer%02d" % i, resource, mu)
        env.process(customer)
        arrival_time = np.random.poisson(interval) # arrivals are a Poisson process
        yield env.timeout(arrival_time)
        i += 1


def serve(env, name, resource, mu):
    """Customer arrives, is served, and leaves
    Args:
        env: Simpy environment
        name: name of customer
        resource: Resource to use
        mu: time it takes to complete a tenant onboarding
    """
    arrive = env.now
    #print("%s arrives at Tenant Onboarding process on day %.2f" % (name, arrive))

    with resource.request() as req:
        yield req
        wait = env.now - arrive
        resource.wait_times.append(wait)
        # Got resource
        #print("%s has begun tenant deployment, waited %.2f day(s)" % (name, wait))
        service_time = np.random.exponential(1/14) # service times are exponential
        service_time += mu  # ensure that the minimum SDP is met
        resource.total_service_time += service_time
        yield env.timeout(service_time)
        resource.customer_times.append((env.now - arrive))
        #print('%s is fully onboarded on day %.2f' % (name, env.now))


if __name__ == "__main__":
    random.seed(RANDOM_SEED)
    env = simpy.Environment()

    res = MonitoredResource(env, capacity=3)
    env.process(arrival(env, ARRIVAL_TIME, TIME_TO_COMPLETE, res))
    env.run(until=SIM_TIME)

    service_time_variability = np.random.exponential(1/TIME_TO_COMPLETE)

    print("\n")
    AVG_WAIT = statistics.mean(res.wait_times)
    print("Expected Average E2E Waiting Time (in days): %.2f" % (AVG_WAIT))
    print("Expected Average Delay in Queue (in days): %.2f" % (AVG_WAIT - (TIME_TO_COMPLETE + service_time_variability)))
    AVG_CUSTOMERS = sum(res.customer_times) / SIM_TIME
    print("Expected Average Number of Customers in Queue: %.2f" % (AVG_CUSTOMERS))
    AVG_UTIL = res.total_service_time / SIM_TIME
    print("Expected Server Utilization: %.2f" % (AVG_UTIL))
    print("\n")



Expected Average E2E Waiting Time (in days): 17.90
Expected Average Delay in Queue (in days): 3.79
Expected Average Number of Customers in Queue: 6.10
Expected Server Utilization: 3.01


